<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class_7-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 7.2**

**Announcements**
* Midterm Lab grades by Friday (drop day)
* Lab 6: Released later today
  * Due Monday (11/7) before class





# Optional stuff

## Interactive Matplotlib

Install `ipympl` for  Matplotlib


In [ ]:
#!pip install ipympl   # Also installs a more recent version of matplotlib (v3.5.3)

Enable interactive Matplotlib figures

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()
# %matplotlib widget

## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
The usual modules...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal
import numpy.fft as fft
import pickle

rc('animation', html='jshtml')

path = '/content/drive/My Drive/eces435-work/class7.2/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'ub40'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helper functions

`myPlot()`

A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)
  * *New* `x_lim=#` or `x_lim=[x1,x2]` Specify the x-axis limit(s) of the plot
  * *New* `y_lim=#` or `y_lim=[y1,y2]` Specificy the y-axis limit(s) of the plot

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt='', x_lim=0, y_lim=0):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = N/fs   # End of input signal
    else:
      x_lim2 = x_lim
    plt.xlim(0, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim != 0:
      plt.ylim(-y_lim, y_lim)
  else:
    plt.ylim(y_lim)

  fig.tight_layout()
  #plt.ion()
  plt.close()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`myPlotFFT()`

Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, y_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_win     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim < 0:
      plt.ylim(y_lim, 0)
    elif y_lim > 0:
      plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB FS)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`mySpectrogram()`

A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.stft(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 4*np.abs(Sxx) + 1e-15
  S_dBFS = 20*np.log10(S_mag)
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  # return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

# Load audio sample

* Happy Halloween!

In [ ]:
x, fs44 = sf.read(path + 'MJ.wav')
ipd.Audio(x, rate=fs44)

# The z-Transform

Just like the Fourier Transform, but with *additional information*.

* Where: $z = r \cdot e^{j\omega}$


$X(z) = \sum_n x[n] z^{-n} = \sum_n x[n] ( r \cdot e^{j\omega})^{-n} = \sum_n r^{-n} x[n] e^{-j\omega n}$





## `zplane()` New function to plot system (filter) information in the $z$-plane.

In [ ]:
def zplane(zeros=[], poles=[], xy_lim=[-1.5,1.5]):
  fig = plt.figure(figsize=(6,6))
  ax = plt.axes()

  # Create radial coordinates as for unit circle contour
  N_pts = 200
  w = np.linspace(0, 2*np.pi, N_pts)
  z_uc = np.exp(1j*w)
  re_uc = np.real(z_uc)
  im_uc = np.imag(z_uc)

  # Draw unit circle and set axis limits
  ax.plot(re_uc, im_uc, 'k:', linewidth=1)
  ax.set_ylim(xy_lim)
  ax.set_xlim(xy_lim)

  # Plot zeros (o) and poles (x)
  re_z = np.real(zeros)
  im_z = np.imag(zeros)
  ax.plot(re_z, im_z, 'bo', markersize=15, fillstyle='none')
  re_p = np.real(poles)
  im_p = np.imag(poles)
  ax.plot(re_p, im_p, 'bx', markersize=13, fillstyle='none')
  
  # X-Y axis labels
  ax.set_xlabel('Real')
  ax.set_ylabel('Imag')

zplane()

# z-Transform of an FIR filter

Basic 2-point moving average filter
* $y[n] = \frac{1}{2}x[n] + \frac{1}{2}x[n-1]$

$z$-Transform
* $Y(z) = \frac{1}{2}X(z) + \frac{1}{2}X(z)z^{-1}$




## How does this help us?

What is the *Transfer Function*, $H(z)$?

* $\frac{Y(z)}{X(z)} = \frac{1}{2}(1 + z^{-1}) = H(z)$

### Where does the transfer function evaluate to...
* zero...?
* infinity...?

$H(z) = \frac{1}{2}(1 + z^{-1}) = \frac{B(z)}{A(z)}$
* $B(z) = \frac{1}{2}(1 + z^{-1})$
* $A(z) = 1$

In [ ]:
zplane(-1)

## What is the response of $H(z)$ on the unit circle? 

In [ ]:
B_z = [0.5, 0.5]
A_z = [1]

w, H_w = signal.freqz(B_z, A_z)
H_dB = 20*np.log10(np.abs(H_w))
plt.plot(w, H_dB)
plt.xlabel('Frequency (rad)')
plt.ylabel('dB')

## But, we're missing a lot of the picture...

### `freqz3D()` A function to visualize the full response across the $z$-plane.

Lots of code here, most of it is to handle the graphics.

In [ ]:
def freqz3D(B=[1], A=[1], azim=-60,
            x_lim=(-1.5,1.5), y_lim=(-1.5,1.5), z_lim=(-30,10),
            x_ticks=(-1,0,1), y_ticks=(-1,0,1),
            fig_size=(14,8), lw=2, xy_spacing=0.005, init_elev=45,
            draw_uc=True, draw_pz=True, draw_contour=True,
            hide_ax=False, hide_z=True, show_fig=True):

  fig = plt.figure(figsize=(14,8))
  ax = plt.axes(projection='3d') 
  z_dB_min = z_lim[0]

  if show_fig==False:
    plt.close()

  # Make cartesian grid in z-plane for freqz surface
  X = np.arange(-1.5, 1.501, xy_spacing) # dB requires different spacing
  Y = np.arange(-1.5, 1.501, xy_spacing) # dB spacing
  X, Y = np.meshgrid(X, Y)
  R = np.sqrt(X**2 + Y**2)
  z = X + Y*1j

  # Create radial coordinates as for unit circle contour(s)
  w = np.linspace(0, 2*np.pi, 200)
  z_c = np.exp(1j*w)
  X_c = np.real(z_c)
  Y_c = np.imag(z_c)

  # If transfer function is provided, plot the freqz surface
  if len(B) > 0:
    Bz =  B[0] * np.ones(np.shape(z))
    Bz_c = B[0] * np.ones(len(z_c))
    for n in range(1,len(B)):
      Bz = Bz + np.divide( B[n], z**n )
      Bz_c = Bz_c + np.divide( B[n], z_c**n )

    if len(A) > 0:
      Az = A[0] * np.ones(np.shape(z))
      Az_c = A[0] * np.ones(len(z_c))
      for n in range(1,len(A)):
        Az = Az + np.divide( A[n], z**n )
        Az_c = Az_c + np.divide( A[n], z_c**n )
    else:
      Az = 1
      Az_c = 1

    Hz = Bz / Az
    Hz_c = Bz_c / Az_c
    Z = np.abs(Hz)
    Z_c = np.abs(Hz_c)
    Z_dB = 20 * np.log10(Z)
    Z_c_dB = 20 * np.log10(Z_c)

    # Plot the freqz surface
    surf = ax.plot_surface(X, Y, Z_dB, color='lightgoldenrodyellow', alpha=0.4, linewidth=0 )
    
  Z_proj = np.zeros(len(w))
  Z_proj_dB = Z_proj + z_lim[0] # include dB offset, depends on z-axis lower limit

  # Draw unit circle at bottom face of z-plane
  if draw_uc:
    ax.plot3D(X_c, Y_c, Z_proj_dB, 'w',linewidth=1)

  # Draw freqz contour on unit circle
  an_object, = ax.plot3D(X_c, Y_c, Z_c_dB, 'y',linewidth=lw)
  if draw_contour == False:
    #Z_c_dB += 200 # Hopefully, +200dB is out of view
    an_object.set_linewidth(0.)

  # Create placeholder animation object for frequency marker
  an_object2, = ax.plot3D([0], [0], [-60], 'w:',linewidth=1)

  if draw_pz:
    if len(B) > 0:
      # Plot pole-zero markers
      zeros,poles,k = signal.tf2zpk(B, A)
      x_z = np.real(zeros)
      y_z = np.imag(zeros)
      x_p = np.real(poles)
      y_p = np.imag(poles)
      z_object, = ax.plot3D(x_z, y_z, z_dB_min*np.ones(len(x_z)), 'wo', markersize=15, fillstyle='none')
      p_object, = ax.plot3D(x_p, y_p, z_dB_min*np.ones(len(x_p)), 'wx', markersize=13, fillstyle='none')
  else:
    # Create empty objects
    z_object, = ax.plot3D([0], [0], [z_dB_min], linewidth=0)
    p_object, = ax.plot3D([0], [0], [z_dB_min], linewidth=0)


  # Set camera view angle
  ax.elev = init_elev
  ax.azim = azim

  # make the panes transparent
  ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
  ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
  ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.05))

  # make the grid lines transparent
  ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
  ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
  ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

  ax.zaxis.set_alpha(0.5)

  ax.set_zlim(z_lim) #dB
  ax.set_xlim(x_lim)
  ax.set_ylim(y_lim)

  if hide_ax:
    ax.set_axis_off()
  else:
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    ax.tick_params(colors='white')
    ax.tick_params(length=10,width=2,labelsize=20)
    for tick in ax.get_xticklabels():
        tick.set_fontname("Liberation Serif")

    for tick in ax.get_yticklabels():
        tick.set_fontname("Liberation Serif")

    fig.patch.set_facecolor('#111111')    # I think this makes the background dark, for preview in Colab

    #ax.set_frame_on(False)
    ax.set_facecolor('#212121')
    ax.get_xaxis().line.set_color('w')
    ax.get_yaxis().line.set_color('w')

    # X-Y axis labels
    ax.set_xlabel('Re', color='w', size=24, fontname='Liberation Serif')
    ax.xaxis.labelpad = 20
    ax.set_ylabel('Im', color='w', size=24, fontname='Liberation Serif')
    ax.yaxis.labelpad = 20

    if hide_z:
      ax.zaxis.line.set_lw(0.)
      ax.set_zticks([])

  fig.tight_layout()

#  if show_fig:
#    plt.show()

  return fig #, ax, an_object, an_object2, surf, p_object, z_object,

### Visualizing the full $z$-plane response

In [ ]:
fig1 = freqz3D(B_z, A_z)

## What's the impulse response of an FIR transfer function?

* $H(z) = 0.5 + 0.5z^{-1}$

* $h[n] = 0.5\delta[n] + 0.5\delta[n-1]$

In [ ]:
H_z = [0.5, 0.5]
d_n = np.zeros(10)
d_n[0] = 1
h_n = signal.lfilter(H_z, 1, d_n)
plt.stem(h_n, basefmt='None')

## Let's examine some more complex FIR filters

In [ ]:
B2_z = np.hanning(9)
B2_z = B2_z / np.sum(B2_z)
h2 = B2_z
plt.stem(h2, basefmt='None')

In [ ]:
fig1 = freqz3D(B2_z, z_lim=[-90,60])

### Factor the numerator of the transfer function, $B(z)$ to determine the location of zeros

In [ ]:
zeros,poles,k = signal.tf2zpk(B2_z, 1)
zplane(zeros, poles)

## How about a filter designed with `firwin()`?

In [ ]:
h3 = signal.firwin(15, 8000, fs=44100)
H3_z = h3
z,p,k = signal.tf2zpk(H3_z,1)
zplane(z,p)

Let's view it from different angles…

In [ ]:
az = -60
fig2 = freqz3D(H3_z, azim=az, z_lim=[-100,40])

In [ ]:
fig2.savefig(path + 'zplane/' + str(az) + username + '.png')

# Can we design a filter by manually placing zeros?

Example: What if we want to eliminate a specific frequency?

In [ ]:
f_notch = 6000      # Frequency of notch
w_notch = (f_notch / fs44) * 2 * np.pi  # Notch frequency in radians
m_notch = 1.0

z_notch = [ m_notch * np.exp(1j*w_notch), m_notch * np.exp(-1j*w_notch) ]
zplane(z_notch, [])

In [ ]:
B_notch, A_notch = signal.zpk2tf(z_notch, [], 1.)
print(B_notch, A_notch)
fig3 = freqz3D(B_notch, A_notch)

In [ ]:
w, H_notch = signal.freqz(B_notch, A_notch)

plt.plot(w, 20*np.log10(np.abs(H_notch)))

In [ ]:
N = len(x)
t = np.arange(N) / fs44
s6k = np.sin(2*np.pi*f_notch*t)
ipd.Audio(s6k, rate=fs44)

In [ ]:
s6k_notch = signal.lfilter(B_notch, A_notch, s6k)
plt.plot(t, s6k_notch)
plt.xlim([0,0.001])

In [ ]:
x_s6k = x + s6k
ipd.Audio(x_s6k, rate=fs44)

In [ ]:
x_notch = signal.lfilter(B_notch, A_notch, x_s6k)
ipd.Audio(x_notch, rate=fs44)

# Why FIR Filters?
Pros:
* Easy to design
* Stable
* Can be linear phase

Cons:
* Need longer filters for sharp transitions
* Sidelobes, never flat or smooth stopband responses

# What about the denominator of that transfer function?

* $H(z) = \frac{1}{1 - a_1 z^{-1}}$

In [ ]:
d_n = np.zeros(100)
d_n[0] = 1.
a_1 = -0.5

B_z = [1]
A_z = [1, a_1]

h_iir = signal.lfilter(B_z, A_z, d_n)
plt.stem(h_iir, basefmt='none')
plt.xlim(0,20)

In [ ]:
w, H_z = signal.freqz(B_z,A_z)
Hz_dB = 20*np.log10(np.abs(H_z))
plt.plot(w, Hz_dB)

In [ ]:
plt.plot(w, np.angle(H_z))

In [ ]:
fig4 = freqz3D(B_z, A_z)

## Why use IIR filters?

Pros:
* Sharper responses with fewer coefficients, due to feedback
* Smooth(er) magnitude response: Can eliminate side lobes

Cons:
* Can be unstable (due to feedback)
* Harder to design



In [ ]:
Z = [1]
P = [0.9*np.exp(1j*0.5), 0.9*np.exp(-1j*0.5)]

zplane(Z,P)

In [ ]:
B, A = signal.zpk2tf(Z, P, 1.)
w, H = signal.freqz(B, A)
H_dB = 20*np.log10(np.abs(H))
fig = plt.figure(figsize=(14,8))
plt.subplot(2,1,1)
plt.plot(w, H_dB)
plt.subplot(2,1,2)
plt.plot(w, np.angle(H))

## Danger!

In [ ]:
d_n = np.zeros(1000)
d_n[0] = 1.

P_u = [1.2*np.exp(1j*0.5), 1.2*np.exp(-1j*0.5)]
zplane([], P_u)

In [ ]:
B_u, A_u = signal.zpk2tf([], P_u, 1.)

h2_iir = signal.lfilter(B_u,A_u,d_n)
plt.plot(h2_iir)

In [ ]:
mags = np.array([0.9, 0.95, 0.75, 0.9])
freqs = np.array([100, 1000, 5000, 10000])
fs = 44100
angles = freqs * 2 * np.pi / (fs)

P1 = np.multiply(mags, np.exp(1j*angles))
P2 = np.multiply(mags, np.exp(-1j*angles))
P = np.append(P1,P2)
Z = [1]
#print(P)
zplane(Z,P)

In [ ]:
print(np.shape(P))
B, A = signal.zpk2tf([1], P, .2)
w, H = signal.freqz(B, A)
H_dB = 20*np.log10(np.abs(H))
fig = plt.figure(figsize=(14,8))
plt.subplot(2,1,1)
plt.plot(w, H_dB)
plt.subplot(2,1,2)
plt.plot(w, np.unwrap(np.angle(H)))

In [ ]:
fig4 = freqz3D(B,A)

In [ ]:
H_dB = 20*np.log10(np.abs(H_notch))
plt.plot(w, H_dB)

In [ ]:
zplane(z_notch, [])

# IIR filter design in the $z$-plane